In [ ]:
# pip install pandas scikit-learn matplotlib seaborn keras tensorflow xgboost lightgbm catboost

  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ----------- ---------------------------- 2.4/8.0 MB 12.2 MB/s eta 0:00:01
   -------------------------- ------------- 5.2/8.0 MB 13.3 MB/s eta 0:00:01
   ---------------------------------------  7.9/8.0 MB 12.8 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 12.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 11.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 2.9/390.3 MB 14.0 MB/s eta 0:00:28
    --------------------------------------- 5.8/390.3 MB 14.1 MB/s eta 0:00:28
    -------------------------------------

In [5]:
import pandas as pd
from sklearn import datasets
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import tensorflow as tf
import xgboost as xgb
import lightgbm as lgb
import catboost as cat

# data_dict_csv = pd.read_csv(r"2025_Problem_C_Data\data_dictionary.csv")
athletes = pd.read_csv(r"2025_Problem_C_Data\summerOly_athletes.csv")
hosts = pd.read_csv(r"2025_Problem_C_Data\summerOly_hosts.csv")
medal = pd.read_csv(r"2025_Problem_C_Data\summerOly_medal_counts.csv")
programs = pd.read_csv(r"2025_Problem_C_Data\summerOly_programs.csv", encoding='windows-1252')

In [48]:
def medal_count_for_each_sport(athletes):
    train = (
        athletes[['Year', 'NOC', 'Sport', 'Event', 'Medal']].drop_duplicates().groupby(['Year', 'NOC', 'Sport', 'Medal'])
        .size()
        .unstack(fill_value=0)
        .reset_index()
    ).sort_values(by=['Year', 'Gold', 'Silver', 'Bronze'], ascending=[False, False, False, False]).loc[:, ['Year', 'NOC', 'Sport', 'Gold', 'Silver', 'Bronze', 'No medal']].reset_index(drop=True)
    return train

def process_hosts(train, hosts):
    country_to_noc = {
        "United States": "USA",
        "United Kingdom": "GBR",
        "France": "FRA",
        "Australia": "AUS",
        "Japan": "JPN",
        "Germany": "GER",
        "Greece": "GRE",
        "Brazil": "BRA",
        "China": "CHN",
        "Spain": "ESP",
        "South Korea": "KOR",
        "Russia": "RUS",
        "Canada": "CAN",
        "Mexico": "MEX",
        "Italy": "ITA",
        "Finland": "FIN",
        "Netherlands": "NED",
        "Belgium": "BEL",
        "Sweden": "SWE",
    }
    hosts['Country'] = hosts['Host'].str.split(',').str[-1].str.strip()
    hosts['NOC'] = hosts['Country'].map(country_to_noc)
    hosts.loc[hosts['Year'] == 1972, 'NOC'] = 'GER'
    hosts.loc[hosts['Year'] == 1980, 'NOC'] = 'RUS'
    hosts.loc[hosts['Year'] == 2020, 'NOC'] = 'JPN'
    hosts = hosts.drop(['Host', 'Country'], axis=1)

    train = pd.merge(train, hosts, on='Year', how='left', suffixes=('', '_host'))
    train['NOC_host'] = train.apply(lambda row: 1 if row['NOC'] == row['NOC_host'] else 0, axis=1)
    return train

def add_EWA_column(train):
    GOLD_WEIGHT = 0.5
    SILVER_WEIGHT = 0.3
    BROUNZE_WEIGHT = 0.2
    beta = 0.8

    train['current_year_value'] = (
        (GOLD_WEIGHT * train['Gold'] + SILVER_WEIGHT * train['Silver'] + BROUNZE_WEIGHT * train['Bronze']) /
        (train['Gold'] + train['Silver'] + train['Bronze'] + train['No medal'])
    )

    train = train.sort_values(by=['NOC', 'Year'])
    train['EWA'] = 0

    for noc, group in train.groupby('NOC'):
        ewa = 0
        t=1
        for idx, row in group.iterrows():
            # Update EWA based on the formula
            ewa = (beta * ewa + (1 - beta) * row['current_year_value'])/(1-beta**t)
            
            train.loc[idx, 'EWA'] = ewa
            t += 1
    train.drop(columns=['current_year_value'], inplace=True)
    return train


In [49]:
train = medal_count_for_each_sport(athletes)


In [50]:
train = process_hosts(train, hosts)

KeyError: 'Host'

In [46]:
train = add_EWA_column(train)

C:\Windows\Temp\ipykernel_28372\312416160.py:64: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.04000002008673564' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train.loc[idx, 'EWA'] = ewa


In [47]:
train.columns

Index(['Year', 'NOC', 'Sport', 'Gold', 'Silver', 'Bronze', 'No medal', 'EWA'], dtype='object', name='Medal')

In [42]:
train.columns
train.head()

Index(['Year', 'NOC', 'Sport', 'Gold', 'Silver', 'Bronze', 'No medal'], dtype='object', name='Medal')

In [ ]:
programs.Sport